
# Volatility Surface from Scratch (Python)
## Option prices → implied volatility (root-finder) → grid → interpolation → interactive surface

This notebook implements the below workflow:

1. Start with **call option market prices** across strikes $K$ and maturities $T$.
2. Implement **Black–Scholes** call pricing from scratch.
3. Compute **implied volatility** $\sigma_{\text{imp}}(K,T)$ by solving:
$
BS(S_0,K,T,r,q,\sigma) = C_{\text{mkt}}(K,T)
$
1. Organize $\sigma_{\text{imp}}(K,T)$ into a **strike-by-maturity grid**.
2. **Interpolate** the grid into a smooth surface you can query at off-grid points.
3. Visualize:
   - Interactive 3D surface
   - Vol Smile/ Skew: $\sigma$ vs $K$ at fixed $T$
   - Term structure: $\sigma$ vs $T$ at fixed $K$

### Important note about “bumps”
If your prices contain noise (bid-ask, stale quotes), the implied vol grid can look bumpy.
This notebook defaults to **zero noise** so the surface is smooth and the smiles look correct.
You can turn noise on later and then increase smoothing.


In [13]:
# %pip install -U numpy pandas scipy plotly

In [14]:
import numpy as np
import pandas as pd

from scipy.stats import norm
from scipy.interpolate import RectBivariateSpline

import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "notebook_connected"
np.random.seed(42)

## 1) Black–Scholes call pricing (from scratch)

### Equations
$$
\begin{aligned}
C &= S_0 e^{-qT} N(d_1) - K e^{-rT} N(d_2)\\[6pt]
d_1 &= \frac{\ln(S_0/K) + \left(r-q + \frac{1}{2}\sigma^2\right)T}{\sigma\sqrt{T}}\\[6pt]
d_2 &= d_1 - \sigma\sqrt{T}
\end{aligned}
$$

### Quant finance meaning
- The market often “quotes” options as **implied volatility** rather than price.
- Black–Scholes converts $\sigma \to C$.
- Implied volatility reverses that: $C_{\text{mkt}} \to \sigma$.

We implement the call price because the implied volatility solver needs repeated pricing calls.


In [15]:
def black_scholes_call(S0: float, K: float, T: float, r: float, sigma: float, q: float = 0.0) -> float:
    '''
    Black-Scholes European call with continuous dividend yield.

    Inputs
    - S0: spot price
    - K: strike
    - T: time to maturity (years)
    - r: risk-free rate
    - q: dividend yield
    - sigma: volatility

    Output
    - Call price
    '''
    if T <= 0.0:
        return max(S0 - K, 0.0)

    # Numerical guard: sigma must be positive
    sigma = max(float(sigma), 1e-12)

    sigma_sqrt_T = sigma * np.sqrt(T)
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma * sigma) * T) / sigma_sqrt_T
    d2 = d1 - sigma_sqrt_T

    discounted_spot = S0 * np.exp(-q * T)
    discounted_strike = K * np.exp(-r * T)

    option_price = float(discounted_spot * norm.cdf(d1) - discounted_strike * norm.cdf(d2))
    return option_price


## 2) Synthetic “market” prices across strikes and maturities

 Here we generate market prices (You can also download market prices yourself here through an API etc.).

We create a “true” volatility surface $\sigma_{\text{true}}(K,T)$ with the classic real-market features:
- **Skew / smile** across strikes (downside tends to have higher implied volatility)
- **Term structure** across maturities

Then we generate market call prices:
$
C_{\text{mkt}}(K,T) = BS(S_0,K,T,r,q,\sigma_{\text{true}}(K,T)) \cdot (1+\varepsilon)
$
where $\varepsilon$ is a small noise term.

To avoid ugly bumps while learning, we default to $\varepsilon = 0$.


In [16]:
# You can also download market prices yourself instead of synthetically generating them!


def sigma_true(S0: float, K: float, T: float) -> float:
    '''
    A smooth, market-like implied vol surface (synthetic):
      - asymmetric skew (downside higher vol)
      - smile curvature (away from ATM higher vol)
      - mild term structure (short-dated slightly higher)
    '''
    m = K / S0  # moneyness

    base = 0.20

    # Smile curvature: stronger for short maturity (1/sqrt(T))
    smile_strength = 0.16 / np.sqrt(np.maximum(T, 1e-6))
    smile = smile_strength * (m - 1.0) ** 2

    # Skew: tilt so low strikes have higher vol
    skew = -0.10 * (m - 1.0)

    # Term structure: extra short-dated vol that decays with T
    term = 0.03 * np.exp(-1.5 * T)

    sig = base + smile + skew + term
    return float(np.clip(sig, 0.05, 0.90))


def generate_synthetic_market_prices(
    S0: float,
    r: float,
    q: float,
    strikes: np.ndarray,
    maturities: np.ndarray,
    noise_std: float = 0.0,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    '''
    Generate a call price table C_mkt(K,T) from sigma_true(K,T).

    Returns
    - market_prices: DataFrame [strike x maturity] of call prices
    - true_vol_grid: DataFrame [strike x maturity] of sigma_true
    '''
    market_prices = pd.DataFrame(index=strikes, columns=maturities, dtype=float)
    true_vol_grid = pd.DataFrame(index=strikes, columns=maturities, dtype=float)

    for K in strikes:
        for T in maturities:
            sig = sigma_true(S0, float(K), float(T))
            C = black_scholes_call(S0, float(K), float(T), r, sig, q=q)

            # Optional multiplicative noise
            eps = np.random.normal(0.0, noise_std) if noise_std > 0 else 0.0
            C_mkt = C * (1.0 + eps)

            # Enforce lower bound: intrinsic under forwards
            intrinsic = max(S0 * np.exp(-q * T) - float(K) * np.exp(-r * T), 0.0)
            C_mkt = max(float(C_mkt), float(intrinsic))

            market_prices.loc[K, T] = C_mkt
            true_vol_grid.loc[K, T] = sig

    return market_prices, true_vol_grid

In [17]:
# ---- Market inputs (illiustrative) ----
S0 = 100.0
r = 0.04
q = 0.00

# ---- Strikes and maturities ----
strikes = np.array([70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 130], dtype=float)
strikes = np.sort(strikes)

maturities = np.array([0.10, 0.25, 0.50, 0.75, 1.00, 1.50, 2.00], dtype=float)
maturities = np.sort(maturities)

# ---- Noise control (keep this at 0.0 while learning) ----
NOISE_STD = 0.00   # try 0.002 or more if you want to see realistic bumps

market_prices,true_vol_grid= generate_synthetic_market_prices(
    S0=S0, r=r, q=q, strikes=strikes, maturities=maturities, noise_std=NOISE_STD
)

print("Market call prices C_mkt(K,T):")
display(market_prices.round(6))


Market call prices C_mkt(K,T):


,0.10,0.25,0.50,0.75,1.00,1.50,2.00
70.0,30.279585,30.712221,31.512460,32.369573,33.243722,34.983994,36.680252
75.0,25.300518,25.795849,26.753227,27.760606,28.765224,30.719388,32.587338
80.0,20.327323,20.941264,22.124423,23.309919,24.454882,26.622640,28.653138
85.0,15.388367,16.235150,17.714543,19.091366,20.373792,22.737260,24.910284
90.0,10.603499,11.837371,13.641316,15.192496,16.590352,19.109439,21.392691
95.0,6.309808,7.978219,10.036983,11.704113,13.173067,15.784296,18.133379
100.0,3.046651,4.892054,7.018128,8.703964,10.180473,12.801068,15.161735
105.0,1.138335,2.701948,4.649481,6.238841,7.650386,10.187991,12.500581
110.0,0.327620,1.344808,2.920230,4.312000,5.591801,7.958001,10.163498
115.0,0.076418,0.611402,1.747730,2.881806,3.982493,6.106407,8.152973



## 3) Implied volatility via a root-finder (bisection)

Implied volatility solves:
$
f(\sigma) = BS(S_0,K,T,r,q,\sigma) - C_{\text{mkt}} = 0
$

### Why bisection is a good default
- For calls, $BS(\sigma)$ increases with $\sigma$.
- This monotonicity makes bisection stable and hard to break.
- Newton’s method is faster but needs extra care (initial guess and Vega logic).


In [18]:
def implied_vol_bisection(
    C_mkt: float,
    S0: float,
    K: float,
    T: float,
    r: float,
    q: float = 0.0,
    sigma_low: float = 1e-8,
    sigma_high: float = 2.0,
    tol: float = 1e-12,
    max_iter: int = 250,
) -> float:
    '''
    Robust implied volatility solver using bisection.
    Returns NaN if a root cannot be bracketed.
    '''
    if T <= 0.0:
        return np.nan

    C_mkt = float(C_mkt)

    def f(sig: float) -> float:
        return black_scholes_call(S0, K, T, r, sig, q=q) - C_mkt

    lo = float(sigma_low)
    hi = float(sigma_high)
    f_lo = f(lo)
    f_hi = f(hi)

    # Expand hi until we bracket a sign change or give up
    tries = 0
    while f_lo * f_hi > 0.0 and tries < 25:
        hi *= 1.5
        f_hi = f(hi)
        tries += 1

    if f_lo * f_hi > 0.0:
        return np.nan

    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        f_mid = f(mid)

        if abs(f_mid) < tol or (hi - lo) < tol:
            return float(mid)

        if f_lo * f_mid <= 0.0:
            hi = mid
            f_hi = f_mid
        else:
            lo = mid
            f_lo = f_mid

    return float(mid)


## 4) Build the implied volatility grid (strike-by-maturity)

For each $(K,T)$, compute:
$
\sigma_{\text{imp}}(K,T) = BS^{-1}(C_{\text{mkt}}(K,T))
$


In [19]:
implied_surface = pd.DataFrame(index=strikes, columns=maturities, dtype=float)

for K in strikes:
    for T in maturities:
        implied_surface.loc[K, T] = implied_vol_bisection(
            C_mkt=float(market_prices.loc[K, T]),
            S0=S0, K=float(K), T=float(T),
            r=r, q=q
        )

print("Implied vol surface recovered from prices (percent):")
display((implied_surface * 100).round(6))

# abs_err = (implied_surface - true_vol_grid).abs()
# print("\nMax absolute vol error (percent points):", float(abs_err.max().max() * 100.0))

Implied vol surface recovered from prices (percent):


,0.10,0.25,0.50,0.75,1.00,1.50,2.00
70.0,30.135804,27.941868,26.453567,25.636726,25.10939,24.491953,24.167595
75.0,28.244402,26.561868,25.331313,24.628658,24.16939,23.632694,23.356468
80.0,26.605982,25.341868,24.322196,23.712966,23.30939,22.838755,22.601910
85.0,25.220544,24.281868,23.426217,22.889650,22.52939,22.110136,21.903920
90.0,24.088088,23.381868,22.643374,22.158709,21.82939,21.446837,21.262498
95.0,23.208615,22.641868,21.973668,21.520145,21.20939,20.848858,20.677645
100.0,22.582124,22.061868,21.417100,20.973957,20.66939,20.316198,20.149361
105.0,22.208615,21.641868,20.973668,20.520145,20.20939,19.848858,19.677645
110.0,22.088088,21.381868,20.643374,20.158709,19.82939,19.446837,19.262498
115.0,22.220544,21.281868,20.426217,19.889650,19.52939,19.110136,18.903920



## 5) Smooth interpolation (bicubic spline) to avoid bumps

Because the data is on a clean grid, **RectBivariateSpline** is a strong default.

We interpolate in $(m,T)$ space:
$
m = \frac{K}{S_0},\quad \sigma = f(m,T)
$

### Smoothing parameter
- `s = 0` means “fit exactly through the grid points” (best when there is no noise).
- If you turn noise on, increase `s` to damp wiggles.


In [20]:
moneyness = (strikes / S0).astype(float)
Z_grid = implied_surface.to_numpy(dtype=float)  # shape (len(moneyness), len(maturities))

SPLINE_SMOOTHING = 0.0 if NOISE_STD == 0.0 else 1e-4

spline = RectBivariateSpline(moneyness, maturities, Z_grid, kx=3, ky=3, s=SPLINE_SMOOTHING)
print("Spline fitted. s =", SPLINE_SMOOTHING)

Spline fitted. s = 0.0



## 6) Interactive 3D surface (smooth)

Axes:
- x: strike $K$ 
- y: time to maturity $T$
- z: implied volatility in percent


In [21]:
# strike: descending
strike_grid = np.linspace(float(strikes.max()), float(strikes.min()), 200)

# time: ascending values
t_grid = np.linspace(float(maturities.min()), float(maturities.max()), 200)

K_grid, T_grid = np.meshgrid(strike_grid, t_grid)
M_grid = K_grid / S0

Z_vol = spline.ev(M_grid.ravel(), T_grid.ravel()).reshape(K_grid.shape)
Z_vol = np.clip(Z_vol, 0.01, 3.0)

surface = go.Surface(
    x=T_grid,               # Time (years)
    y=K_grid,               # Strike
    z=Z_vol * 100.0,        # Vol (%)
    opacity=0.92,
    colorbar=dict(title="Implied vol (percent)", len=0.75),
    name="Spline surface",
)

# Scatter points (order doesn't matter for markers)
dots_x, dots_y, dots_z = [], [], []
for T in np.sort(maturities):                 # keep time ascending in points too
    for K in strikes:
        dots_x.append(float(T))
        dots_y.append(float(K))
        dots_z.append(float(implied_surface.loc[K, T]) * 100.0)

dots = go.Scatter3d(
    x=dots_x, y=dots_y, z=dots_z,
    mode="markers",
    marker=dict(size=4),
    name="IV grid points",
)

fig = go.Figure(data=[surface, dots])
fig.update_layout(
    title=dict(
        text="Implied Volatility Surface (x=Time, y=Strike, z=Vol)",
        x=0.5, xanchor="center",
        pad=dict(t=2, b=2),
    ),
    height=820,
    width=1120,

    # Remove whitespace around the figure
    margin=dict(l=0, r=0, t=35, b=0),

    scene=dict(
        # Make the 3D scene use the full canvas
        domain=dict(x=[0.0, 1.0], y=[0.0, 1.0]),

        # Time axis: show 0.5 at the "start" by reversing visual direction
        xaxis=dict(title="Time to maturity (years)", autorange="reversed"),

        yaxis=dict(title="Strike (K)"),
        zaxis=dict(title="Implied volatility (percent)"),

        # Optional: zoom camera a bit so plot fills space better
        camera=dict(eye=dict(x=1.35, y=1.35, z=0.9)),
    ),
)

fig.show()



## 7) Smile and term structure 
If `NOISE_STD = 0.0`, these curves should be smooth.
If you add noise, you will see wiggles. 


In [22]:
# Smile: implied volatility versus strike for each maturity
figure_smile = go.Figure()

for t in implied_surface.columns:
    figure_smile.add_trace(
        go.Scatter(
            x=implied_surface.index.to_numpy(dtype=float),
            y=(implied_surface[t].to_numpy(dtype=float) * 100.0),
            mode="lines+markers",
            name=f"Time to maturity = {float(t):.2f} years",
        )
    )

figure_smile.add_vline(x=S0, line_dash="dash")
figure_smile.update_layout(
    title="Volatility Smile (Implied volatility versus strike)",
    xaxis_title="Strike",
    yaxis_title="Implied volatility (percent)",
    height=480,
)
figure_smile.show()


# Term structure: implied volatility versus maturity for selected strikes near spot
strike_array = implied_surface.index.to_numpy(dtype=float)
atm_index = int(np.argmin(np.abs(strike_array - S0)))
selected_strikes = strike_array[max(0, atm_index - 2): min(len(strike_array), atm_index + 3)]

figure_term = go.Figure()
for k in selected_strikes:
    figure_term.add_trace(
        go.Scatter(
            x=implied_surface.columns.to_numpy(dtype=float),
            y=(implied_surface.loc[k].to_numpy(dtype=float) * 100.0),
            mode="lines+markers",
            name=f"Strike = {k:.0f}",
        )
    )

figure_term.update_layout(
    title="Volatility Term Structure (Implied volatility versus maturity)",
    xaxis_title="Time to maturity (years)",
    yaxis_title="Implied volatility (percent)",
    height=480,
)
figure_term.show()


## 8) Sanity check: implied vol reprices the market

You can verify the inversion worked by repricing:
$
\hat{C}(K,T) = BS(S_0,K,T,r,q,\sigma_{\text{imp}}(K,T))
$
and comparing $\hat{C}$ to $C_{\text{mkt}}$.


In [23]:
repriced = pd.DataFrame(index=strikes, columns=maturities, dtype=float)

for K in strikes:
    for T in maturities:
        sig = float(implied_surface.loc[K, T])
        repriced.loc[K, T] = black_scholes_call(S0, float(K), float(T), r, sig, q=q)

abs_price_err = (repriced - market_prices).abs()
print("Max absolute repricing error:", float(abs_price_err.max().max()))
display(abs_price_err.round(12))

Max absolute repricing error: 2.4314772417710628e-11


,0.10,0.25,0.50,0.75,1.00,1.50,2.00
70.0,1.000000e-12,0.000000e+00,1.000000e-12,1.000000e-12,0.000000e+00,1.000000e-12,2.000000e-12
75.0,0.000000e+00,1.000000e-12,1.000000e-12,1.000000e-12,2.000000e-12,3.000000e-12,1.000000e-12
80.0,0.000000e+00,0.000000e+00,1.000000e-12,2.000000e-12,1.000000e-12,3.000000e-12,9.000000e-12
85.0,1.000000e-12,1.000000e-12,0.000000e+00,4.000000e-12,0.000000e+00,4.000000e-12,1.000000e-11
90.0,1.000000e-12,1.000000e-12,2.000000e-12,9.000000e-12,1.100000e-11,1.000000e-12,1.700000e-11
95.0,1.000000e-12,4.000000e-12,5.000000e-12,1.000000e-11,9.000000e-12,1.000000e-12,3.000000e-12
100.0,0.000000e+00,5.000000e-12,8.000000e-12,1.000000e-12,1.300000e-11,1.700000e-11,9.000000e-12
105.0,3.000000e-12,0.000000e+00,1.000000e-12,1.200000e-11,1.600000e-11,1.100000e-11,1.000000e-11
110.0,1.000000e-12,0.000000e+00,9.000000e-12,4.000000e-12,4.000000e-12,1.000000e-12,4.000000e-12
115.0,1.000000e-12,2.000000e-12,4.000000e-12,1.200000e-11,1.000000e-11,1.600000e-11,2.400000e-11



## 9) Export to CSV (optional)


In [24]:
market_prices.to_csv("market_call_prices.csv")
implied_surface.to_csv("implied_vol_surface.csv")

print("Saved: market_call_prices.csv")
print("Saved: implied_vol_surface.csv")

Saved: market_call_prices.csv
Saved: implied_vol_surface.csv
